In [12]:
import os

In [13]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection/research'

In [14]:
os.chdir("../")

In [15]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

In [16]:
from dataclasses import dataclass
from pathlib import Path

In [17]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    A class for holding an data path
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
    


In [18]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml,create_directories

In [19]:
print(CONFIG_FILE_PATH)

config/config.yaml


In [20]:
print(PARAMS_FILE_PATH)

params.yaml


In [21]:
print(SCHEMA_FILE_PATH)

schema.yaml


In [22]:
print("hi")

hi


In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_file_path),
        self.params = read_yaml(params_file_path),
        self.schema = read_yaml(schema_file_path)
    
        print(self.config)
        print(self.config[0].artifacts_root)
        create_directories([self.config[0].artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config[0].data_ingestion
        create_directories([config.root_dir])   
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
         )
        
        return data_ingestion_config

 

In [24]:
import os
import urllib.request as request
import zipfile
from Credit_Card_Fraud_Detection import logger
from Credit_Card_Fraud_Detection.utils.common import get_size

In [25]:
class DataIngestionStrategy:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        print("="*50)
        print(self.config.local_data_file)
        print("="*50)
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  
        


In [26]:
# Pipelines for the DataIngestionStrategy
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestionStrategy(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-01-04 22:08:53,081: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-04 22:08:53,084: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-04 22:08:53,088: INFO: common: yaml file: schema.yaml loaded successfully]
(ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection/raw/main/data/credit_card_dataset.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/creditcard.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/creditcard.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/

URLError: <urlopen error [Errno 60] Operation timed out>